In [ ]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import requests
import mysql.connector
import numpy as np
from datetime import datetime

In [ ]:
Comps = ["MSFT", "AAPL", "NVDA", "AVGO", "BRK-B", "JPM",
         "LLY", "UNH", "AMZN", "GE", "GOOG", "WMT", "XOM", "LIN",]

nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/bcalinat/nltk_data...


In [ ]:
# newsapi return example:
# {'status': 'ok',
#  'totalResults': 123,
#  'articles': [{'source': {'id': None, 'name': 'Biztoc.com'},
#    'author': 'investors.com',
#    'title': "xxxxxx",
#    'description': 'xxxxx',
#    'url': 'https://xxxxx',
#    'urlToImage': 'https://xxxxxxx',
#    'publishedAt': '2024-03-01T23:28:14Z',
#    'content': "xxxxx"},
#  ]
# }

# use example -> get_news("MSFT", "2024-03-01", "2022-04-01")
def get_news_by_newsapi(query: str, fromTime: str, toTime: str):
    api_key = '490c4bd988b94431886ee1bd945a5e11'
    url = 'https://newsapi.org/v2/everything'
    params = {
        'q': query,
        'from': fromTime,
        'to': toTime,
        'sortBy': 'publishedAt',
        'apiKey': api_key
    }
    response = requests.get(url, params=params)
    data = response.json()
    # print(data)
    resp = []
    for i in data['articles']:
        resp.append({
			'content': i['content'],
            'url': i['url'],
			'publishedAt': i['publishedAt'],
            'title': i['title']
        })
    return resp

In [ ]:
# get_score return example: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
# compound 是最终打分，大于0为正面，小于0为负面
def get_score(text: str):
    score = sia.polarity_scores(text)
    return score

In [ ]:
# get_month_news_for_company: 对股票(公司)名，获取从202X到202X年每个月的新闻打分细节
def get_month_news_for_company(compName: str, startYear: int, endYear: int):
	news = []
	for y in range(startYear, endYear + 1): # 遍历从202X到202X年
		for m in range(1, 12): # 遍历12个月
            #
            # 
			comp_news = get_news_by_newsapi(compName, f'{y}-{m}-1', f'{y}-{m+1}-1')
			for artical in comp_news['articles']:
				score = get_score(artical['content']) 
				news.append({
					'name': compName,
					'url': artical['url'],
					'publishedAt': artical['publishedAt'],
					'compound': score['compound'],
				})
	return news

# 获取所有公司的记录
def get_month_news_for_companies(comps: list, startYear: int, endYear: int):
    return [get_month_news_for_company(comp, startYear, endYear) for comp in comps]